In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import cv2

# ==============================
# SETTINGS
# ==============================
DATASET_PATH = "/kaggle/input/datasets/nodoubttome/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration"

IMG_SIZE = 300
BATCH_SIZE = 16
EPOCHS_HEAD = 15
EPOCHS_FINE = 10
NUM_CLASSES = 9

# ==============================
# FOCAL LOSS
# ==============================
def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1. - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        return tf.reduce_sum(weight * cross_entropy, axis=1)
    return loss

# ==============================
# LOAD ALL IMAGE PATHS
# ==============================
image_paths = []
labels = []

for split in ["Train", "Test"]:
    split_path = os.path.join(DATASET_PATH, split)
    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)
        for img in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img))
            labels.append(class_name)

image_paths = np.array(image_paths)
labels = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

# ==============================
# TRAINING LOOP
# ==============================
for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels_encoded)):

    print(f"\n🔥 Training Fold {fold+1}")

    train_paths, val_paths = image_paths[train_idx], image_paths[val_idx]
    train_labels, val_labels = labels_encoded[train_idx], labels_encoded[val_idx]

    def load_data(paths, labels):
        images = []
        for path in paths:
            img = cv2.imread(path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
        images = np.array(images)
        labels = keras.utils.to_categorical(labels, NUM_CLASSES)
        return images, labels

    X_train, y_train = load_data(train_paths, train_labels)
    X_val, y_val = load_data(val_paths, val_labels)

    # ------------------ MODEL ------------------
    base_model = EfficientNetB3(
        weights="imagenet",
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = preprocess_input(inputs)
    x = layers.RandomFlip("horizontal")(x)
    x = layers.RandomRotation(0.1)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss=focal_loss(),
        metrics=["accuracy"]
    )

    # ------------------ TRAIN HEAD ------------------
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS_HEAD,
        batch_size=BATCH_SIZE,
        verbose=1
    )

    # ------------------ FINE TUNE LAST 40 LAYERS ------------------
    for layer in base_model.layers[-40:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    model.compile(
        optimizer=keras.optimizers.Adam(5e-6),
        loss=focal_loss(),
        metrics=["accuracy"]
    )

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS_FINE,
        batch_size=BATCH_SIZE,
        verbose=1
    )

    loss, acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")

    fold_accuracies.append(acc)

    model.save(f"final_fold_{fold+1}.keras")

# ==============================
# FINAL RESULT
# ==============================
print("\n📊 Fold Accuracies:", fold_accuracies)
print("🔥 Average Accuracy:", np.mean(fold_accuracies))


2026-02-17 03:50:52.193642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771300252.380426      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771300252.435134      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771300252.882164      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771300252.882204      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771300252.882206      55 computation_placer.cc:177] computation placer alr


🔥 Training Fold 1


I0000 00:00:1771300311.137780      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1771300311.143742      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15


E0000 00:00:1771300329.512129      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1771300332.377624     124 cuda_dnn.cc:529] Loaded cuDNN version 91002


...

118/118 ━━━━━━━━━━━━━━━━━━━━ 44s 222ms/step - accuracy: 0.6699 - loss: 0.1267 - val_accuracy: 0.5605 - val_loss: 0.2062
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 23s 197ms/step - accuracy: 0.6923 - loss: 0.1243 - val_accuracy: 0.5563 - val_loss: 0.2044
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 181ms/step - accuracy: 0.6908 - loss: 0.1226 - val_accuracy: 0.5499 - val_loss: 0.2061
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 179ms/step - accuracy: 0.7108 - loss: 0.1171 - val_accuracy: 0.5499 - val_loss: 0.2064
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 22s 187ms/step - accuracy: 0.7054 - loss: 0.1134 - val_accuracy: 0.5563 - val_loss: 0.2046
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 22s 189ms/step - accuracy: 0.6847 - loss: 0.1212 - val_accuracy: 0.5563 - val_loss: 0.2003
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 181ms/step - accuracy: 0.7002 - loss: 0.1170 - val_accuracy: 0.5605 - val_loss: 0.2065
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 182ms/step - accuracy: 0.7083 - loss: 0.1123 - val

E0000 00:00:1771302693.106659      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_4_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


118/118 ━━━━━━━━━━━━━━━━━━━━ 38s 202ms/step - accuracy: 0.2445 - loss: 0.3884 - val_accuracy: 0.4034 - val_loss: 0.2905
Epoch 2/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 180ms/step - accuracy: 0.4372 - loss: 0.2782 - val_accuracy: 0.4628 - val_loss: 0.2590
Epoch 3/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 20s 166ms/step - accuracy: 0.4999 - loss: 0.2323 - val_accuracy: 0.5074 - val_loss: 0.2367
Epoch 4/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 19s 161ms/step - accuracy: 0.5053 - loss: 0.2251 - val_accuracy: 0.5032 - val_loss: 0.2285
Epoch 5/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 20s 165ms/step - accuracy: 0.5163 - loss: 0.2103 - val_accuracy: 0.5414 - val_loss: 0.2107
Epoch 6/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 20s 173ms/step - accuracy: 0.5444 - loss: 0.1998 - val_accuracy: 0.5584 - val_loss: 0.2045
Epoch 7/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 20s 166ms/step - accuracy: 0.5615 - loss: 0.1856 - val_accuracy: 0.5584 - val_loss: 0.1993
Epoch 8/15
118/118 ━━━━━━━━━━━━━━━━━━━━ 19s 164ms/step - accuracy: 0.5739 - loss: 0.1804 - val

E0000 00:00:1771303011.686136      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_4_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


118/118 ━━━━━━━━━━━━━━━━━━━━ 44s 221ms/step - accuracy: 0.6383 - loss: 0.1431 - val_accuracy: 0.6136 - val_loss: 0.1787
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 23s 197ms/step - accuracy: 0.6944 - loss: 0.1264 - val_accuracy: 0.6072 - val_loss: 0.1791
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 181ms/step - accuracy: 0.6733 - loss: 0.1226 - val_accuracy: 0.6136 - val_loss: 0.1767
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 178ms/step - accuracy: 0.7067 - loss: 0.1137 - val_accuracy: 0.6051 - val_loss: 0.1792
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 22s 187ms/step - accuracy: 0.6863 - loss: 0.1138 - val_accuracy: 0.6030 - val_loss: 0.1749
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 22s 187ms/step - accuracy: 0.6943 - loss: 0.1120 - val_accuracy: 0.6072 - val_loss: 0.1766
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 21s 182ms/step - accuracy: 0.7092 - loss: 0.1106 - val_accuracy: 0.6115 - val_loss: 0.1741
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 22s 183ms/step - accuracy: 0.6926 - loss: 0.1147 - val